Import keras

In [41]:
import tensorflow as tf
from keras import Input
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [50]:
max_words = 10000  # Chỉ lấy 10,000 từ phổ biến nhất
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words)

print(f"Số mẫu train: {len(x_train)}")  # 25,000
print(f"Số mẫu test: {len(x_test)}")    # 25,000
print(f"Nhãn mẫu đầu tiên: {y_train[0]}")

Số mẫu train: 25000
Số mẫu test: 25000
Nhãn mẫu đầu tiên: 1


In [51]:
max_len = 200

Pre

In [52]:
x_train_padded = pad_sequences(x_train, maxlen=max_len, padding='post', truncating='post')
x_test_padded = pad_sequences(x_test, maxlen=max_len, padding='post', truncating='post')

Make embedding layer from pretrain glove

Read the downloaded glove file

In [54]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [55]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('./data/glove.6B.50d.txt')


Make embedding layer

In [58]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index, max_words):
    vocab_len = max_words + 1  # +1 cho padding token
    emb_dim = word_to_vec_map["father"].shape[0]
    emb_matrix = np.zeros((vocab_len, emb_dim))

    imdb_word_index = imdb.get_word_index()
    for word, idx in imdb_word_index.items():
        if idx < max_words and word in word_to_vec_map:
            emb_matrix[idx + 1] = word_to_vec_map[word]  # +1 vì IMDB index bắt đầu từ 1

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [59]:
pretrained_embedding_layer(word_to_vec_map, word_to_index, max_words)

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


<Embedding name=embedding_6, built=True>

Building model

In [63]:
def Imdb_classifation(input_shape, max_words, word_to_vec_map, word_to_index):

    sentence_indices = Input(input_shape, dtype='int32')

    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index, max_words)

    embeddings = embedding_layer(sentence_indices) # get the embedding of the sentence

    X = LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    X = Dense(1)(X)
    # Add a softmax activation
    X = Activation('sigmoid')(X)

    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)

    return model


In [64]:
model = Imdb_classifation((max_len,), max_words, word_to_vec_map, word_to_index)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_7 (Embedding)         │ (None, 200, 50)        │       500,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 200, 128)       │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 200, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 723,411 (2.76 MB)

 Trainable params: 223,361 (872.50 KB)

 Non-trainable params: 500,050 (1.91 MB)

In [65]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train_padded, y_train, epochs=20, batch_size=32, shuffle=True,
          validation_data=(x_test_padded, y_test))

Epoch 1/20
407/782 ━━━━━━━━━━━━━━━━━━━━ 1:01 163ms/step - accuracy: 0.5027 - loss: 0.6983

In [ ]:
loss, accuracy = model.evaluate(x_test_padded, y_test)
print(f"Test accuracy: {accuracy}")